# Описание проекта

Провести оценку результатов A/B-теста

- Оценить корректность проведения теста
- Проанализировать  результаты теста

Чтобы оценить корректность проведения теста, проверьте:

- пересечение тестовой аудитории с конкурирующим тестом,
- совпадение теста и маркетинговых событий, другие проблемы временных границ теста.

# Техническое задание

- Название теста: `recommender_system_test`;
- группы: А — контрольная, B — новая платёжная воронка;
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
- дата остановки: 2021-01-04;
- аудитория: 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.

# План работ

1. <a href='#1'> Изучение файлов с данными, получение общей информации, загрузка библиотек  </a>
- Проверить пропуски и типы данных. 
- Проверить дубликаты


2. <a href='#2'> Оценить корректность проведения теста </a>

  - Соответствие данных требованиям технического задания. Проверьте корректность всех пунктов технического задания.
  - Время проведения теста. Убедитесь, что оно не совпадает с маркетинговыми и другими активностями.
  - Аудиторию теста. Удостоверьтесь, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверьте равномерность распределения по тестовым группам и правильность их формирования.

3. <a href='#3'> Исследовательский анализ данных </a>
- Количество событий на пользователя одинаково распределены в выборках?
- Как число событий в выборках распределено по дням?
- Как меняется конверсия в воронке в выборках на разных этапах?
- Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?



4. <a href='#4'> Оценить результаты A/B-тестирования </a>
- Что можно сказать про результаты A/В-тестирования?
- Проверьте статистическую разницу долей z-критерием.

5.<a href='#5'> Выводы</a>

 Сделать общее заключение о корректности проведения теста. 


<a id='1'></a>
### Изучение файлов с данными, получение общей информации, загрузка библиотек 

In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime as dt
import numpy as np
import math as mth
from scipy import stats as st

import plotly.express as px 
from plotly import graph_objects as go
import seaborn as sns

pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_105/509102035.py:13: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



In [56]:
def request(df):
    return pd.read_csv(df, sep=',')

In [57]:
final_ab_new_users=request('/datasets/final_ab_new_users.csv')
final_ab_events=request('/datasets/final_ab_events.csv')
final_ab_participants=request('/datasets/final_ab_participants.csv')
final_ab_marketing_events=request('/datasets/ab_project_marketing_events.csv')

In [58]:
def inspect(df):
    display(df.head()) 
    print ('Количество дубликатов -', df.duplicated().sum())
    numeric_columns=df.describe().columns
    display(df.describe())
    display()
    #orders.columns = [x.lower().replace(' ', '_') for x in df.columns.values]
    for column_name in df.columns:
        if 'first_date' in column_name or 'date' in column_name:
                df[column_name] = df[column_name].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
        else:
                display(df[column_name].value_counts())
                print(50*'*')
    df.info() 
    return df
final_ab_new_users = inspect(final_ab_new_users)


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


Количество дубликатов - 0


,user_id,first_date,region,device
count,61733,61733,61733,61733
unique,61733,17,4,4
top,A900A8E3B077CD75,2020-12-21,EU,Android
freq,1,6290,46270,27520


A900A8E3B077CD75    1
7D82557D75E36086    1
CA8E03E71A8400D3    1
A0A3E7A9437437A9    1
EEBD18FF754F6C98    1
                   ..
21843F4B10A163EA    1
133551F803A76F52    1
D59BFC277B8630CB    1
AAF2AE5411A16B7E    1
D8F5EF12AD39E391    1
Name: user_id, Length: 61733, dtype: int64

**************************************************


EU           46270
N.America    9155 
CIS          3155 
APAC         3153 
Name: region, dtype: int64

**************************************************


Android    27520
PC         15599
iPhone     12530
Mac        6084 
Name: device, dtype: int64

**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     61733 non-null  object        
 1   first_date  61733 non-null  datetime64[ns]
 2   region      61733 non-null  object        
 3   device      61733 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ MB


In [59]:
final_ab_participants = inspect(final_ab_participants)

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


Количество дубликатов - 0


,user_id,group,ab_test
count,18268,18268,18268
unique,16666,2,2
top,5438D422A6A53E0E,A,interface_eu_test
freq,2,9655,11567


5438D422A6A53E0E    2
EB87A4B3C0008AFB    2
32AF1446FF633378    2
70BF82527E6ED9C3    2
C10DF84409319DD3    2
                   ..
C323F544EF1B9E45    1
CDFB5EAA70A846E8    1
CBB3A0A3F97BEC52    1
C7D13A92995405A2    1
C3052D5A747E9CD8    1
Name: user_id, Length: 16666, dtype: int64

**************************************************


A    9655
B    8613
Name: group, dtype: int64

**************************************************


interface_eu_test          11567
recommender_system_test    6701 
Name: ab_test, dtype: int64

**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


In [60]:
final_ab_events = inspect(final_ab_events)

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


Количество дубликатов - 0


,details
count,62740.000000
mean,23.877631
std,72.180465
min,4.990000
25%,4.990000
50%,4.990000
75%,9.990000
max,499.990000


A3917F81482141F2    36
590DACD07A839BC3    32
8AEB716C4CD82CA7    32
426330B820B8711A    32
11285A53EC2B2042    32
                    ..
B4A4C1152C7A6CE9    1 
788724F2A10C59FE    1 
85A8CCD5C08A18E4    1 
6D6D3E965F77305D    1 
812A667823327EDD    1 
Name: user_id, Length: 58703, dtype: int64

**************************************************


...

**************************************************


login           189552
product_page    125563
purchase        62740 
product_cart    62462 
Name: event_name, dtype: int64

**************************************************


4.99      46362
9.99      9530 
99.99     5631 
499.99    1217 
Name: details, dtype: int64

**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


In [61]:
final_ab_marketing_events = inspect(final_ab_marketing_events)

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


Количество дубликатов - 0


,name,regions,start_dt,finish_dt
count,14,14,14,14
unique,14,6,14,14
top,CIS New Year Gift Lottery,APAC,2020-12-25,2020-05-03
freq,1,4,1,1


CIS New Year Gift Lottery           1
Chinese New Year Promo              1
Chinese Moon Festival               1
Easter Promo                        1
Dragon Boat Festival Giveaway       1
International Women's Day Promo     1
Single's Day Gift Promo             1
Labor day (May 1st) Ads Campaign    1
St. Patric's Day Promo              1
Christmas&New Year Promo            1
Black Friday Ads Campaign           1
St. Valentine's Day Giveaway        1
4th of July Promo                   1
Victory Day CIS (May 9th) Event     1
Name: name, dtype: int64

**************************************************


...

**************************************************


2020-12-25    1
2020-07-04    1
2020-11-26    1
2020-02-14    1
2020-03-17    1
2020-12-30    1
2020-06-25    1
2020-03-08    1
2020-10-01    1
2020-05-09    1
2020-11-11    1
2020-05-01    1
2020-04-12    1
2020-01-25    1
Name: start_dt, dtype: int64

**************************************************


2020-05-03    1
2020-05-11    1
2020-10-07    1
2021-01-07    1
2020-11-12    1
2021-01-03    1
2020-04-19    1
2020-02-16    1
2020-07-01    1
2020-03-19    1
2020-03-10    1
2020-12-01    1
2020-07-11    1
2020-02-07    1
Name: finish_dt, dtype: int64

**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


Дубликатов в данных не обнаружено

Даты привели к нужному  типу 

Обнаружены пропуски в таблице final_ab_events в столбце details. Так как этот столбец содержит всего лишь допол. информацию, пропуски оставим как есть.

В столбце ab_test из таблицы final_ab_participants , возможно, остались данные из другого теста. Необходимо удалить.

<a id='2'></a>
### Оценить корректность проведения теста 

 - Соответствие данных требованиям технического задания. Проверьте корректность всех пунктов технического задания.
  - Время проведения теста. Убедитесь, что оно не совпадает с маркетинговыми и другими активностями.
  - Аудиторию теста. Удостоверьтесь, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверьте равномерность распределения по тестовым группам и правильность их формирования.

#### Соответствие данных требованиям технического задания

Необходимо проверить данные по ТЗ:

    - название теста: recommender_system_test
    - пользователи, зарегистрированные до 2020-12-21
    - дата проведения теста с 2020-12-07 по 2021-01-04
    - аудитория: 15% новых пользователей из региона EU

In [62]:
final_ab_participants

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test
...,...,...,...
18263,1D302F8688B91781,B,interface_eu_test
18264,3DE51B726983B657,A,interface_eu_test
18265,F501F79D332BE86C,A,interface_eu_test
18266,63FBE257B05F2245,A,interface_eu_test


In [63]:
#проверим группы на пересекающихся пользователей
duplicated_users = final_ab_participants.groupby('user_id').agg({'group': ['nunique', 'unique']})
duplicated_users.columns = ['groups', 'group_names']
duplicated_users = duplicated_users.query('groups > 1')
display(len(duplicated_users))

776

In [64]:
final_ab_participants['ab_test'].value_counts()

interface_eu_test          11567
recommender_system_test    6701 
Name: ab_test, dtype: int64

Как видим, наших только 6701 пользователей, необходимо оставить нужных нам пользователей.


In [65]:


temp = final_ab_participants.groupby('user_id').agg({'ab_test': 'nunique'}).reset_index()
temp.query('ab_test > 1')

,user_id,ab_test
2,001064FEAAB631A1,2
10,00341D8401F0F665,2
12,003B6786B4FF5B03,2
29,0082295A41A867B5,2
46,00E68F103C66C1F7,2
...,...,...
16638,FF7BE2897FC0380D,2
16644,FF9A81323FA67D6E,2
16652,FFC53FD45DDA5EE8,2
16662,FFED90241D04503F,2


In [66]:
new_ab_participants=final_ab_participants.query('ab_test=="recommender_system_test"')
new_ab_participants

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test
...,...,...,...
6696,053FB26D6D49EDDC,A,recommender_system_test
6697,9D263B8EF15CF188,B,recommender_system_test
6698,F2FBBA33F37DEC46,A,recommender_system_test
6699,29C92313A98B1176,B,recommender_system_test


In [67]:
final_ab_new_users


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
61728,1DB53B933257165D,2020-12-20,EU,Android
61729,538643EB4527ED03,2020-12-20,EU,Mac
61730,7ADEE837D5D8CBBD,2020-12-20,EU,PC
61731,1C7D23927835213F,2020-12-20,EU,iPhone


In [68]:
#найдем максимальную и минимальную дату:
print("Первая дата регистрации пользователей : {}".format(final_ab_new_users['first_date'].min()))
print("Последняя дата регистрации пользователей : {}".format(final_ab_new_users['first_date'].max()))

Первая дата регистрации пользователей : 2020-12-07 00:00:00
Последняя дата регистрации пользователей : 2020-12-23 00:00:00


Согласно ТЗ, срок остановки набора новых пользователей 2020-12-21, поэтому всех кто после этой даты зарегистрировался уберем.

In [69]:
final_ab_new_users

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
61728,1DB53B933257165D,2020-12-20,EU,Android
61729,538643EB4527ED03,2020-12-20,EU,Mac
61730,7ADEE837D5D8CBBD,2020-12-20,EU,PC
61731,1C7D23927835213F,2020-12-20,EU,iPhone


In [70]:
new_final_ab_new_users=final_ab_new_users.query('first_date<="2020-12-21"')
new_final_ab_new_users

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
61728,1DB53B933257165D,2020-12-20,EU,Android
61729,538643EB4527ED03,2020-12-20,EU,Mac
61730,7ADEE837D5D8CBBD,2020-12-20,EU,PC
61731,1C7D23927835213F,2020-12-20,EU,iPhone


In [71]:
print("Убрали {} пользователей ".format(len(final_ab_new_users['first_date'])-len(new_final_ab_new_users['first_date'])))

Убрали 5263 пользователей 


Проверим дату проведения теста

In [72]:
final_ab_events

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
...,...,...,...,...
440312,245E85F65C358E08,2020-12-30 19:35:55,login,NaN
440313,9385A108F5A0A7A7,2020-12-30 10:54:15,login,NaN
440314,DB650B7559AC6EAC,2020-12-30 10:59:09,login,NaN
440315,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,NaN


In [73]:
#найдем максимальную и минимальную дату:
print("Первая дата регистрации пользователей : {}".format(final_ab_events['event_dt'].min()))
print("Последняя дата регистрации пользователей : {}".format(final_ab_events['event_dt'].max()))

Первая дата регистрации пользователей : 2020-12-07 00:00:33
Последняя дата регистрации пользователей : 2020-12-30 23:36:33


Согласно ТЗ, дата проведения теста с 2020-12-07 по 2021-01-04, данные только до 30.12.2020.

Оставим только новых пользователей из региона EU

In [74]:
new_final_ab_new_users

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
61728,1DB53B933257165D,2020-12-20,EU,Android
61729,538643EB4527ED03,2020-12-20,EU,Mac
61730,7ADEE837D5D8CBBD,2020-12-20,EU,PC
61731,1C7D23927835213F,2020-12-20,EU,iPhone


In [75]:
users_from_EU=new_final_ab_new_users.query('region=="EU"')
users_from_EU

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
7,8942E64218C9A1ED,2020-12-07,EU,PC
9,FFCEA1179C253104,2020-12-07,EU,Android
...,...,...,...,...
61728,1DB53B933257165D,2020-12-20,EU,Android
61729,538643EB4527ED03,2020-12-20,EU,Mac
61730,7ADEE837D5D8CBBD,2020-12-20,EU,PC
61731,1C7D23927835213F,2020-12-20,EU,iPhone


In [76]:
print("Убрали {} пользователей ".format(len(new_final_ab_new_users['user_id'].unique())-len(users_from_EU['user_id'].unique())))

Убрали 14130 пользователей 


Проверим какая фактическая доля пользователей в тесте по региону EU относительно всех новых пользователей в final_ab_new_users по региону EU и времени регистрации в тесте.

In [77]:
dd=new_ab_participants['user_id'].unique()
dd

array(['D1ABA3E2887B6A73', 'A7A3664BD6242119', 'DABC14FDDFADD29E', ...,
       'F2FBBA33F37DEC46', '29C92313A98B1176', '6715343AFBA285AE'],
      dtype=object)

In [78]:
f=users_from_EU.query('user_id in @dd')

In [79]:
print("Доля пользователей в тесте по региону EU относительно всех новых пользователей по региону EU -{} ".format(round((users_from_EU.query('user_id in @dd')['user_id'].nunique()/users_from_EU['user_id'].nunique())*100,2)))

Доля пользователей в тесте по региону EU относительно всех новых пользователей по региону EU -15.0 


Согласно ТЗ, aудитория из новых пользователей из региона EU должна составлять 15% , по расчетам примерно 14%.

Проверим какие мероприятия, проходили в период действия теста, дата проведения теста с 2020-12-07 по 2021-01-04.

In [80]:
final_ab_marketing_events.head()

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


In [81]:
final_ab_marketing_events.query('start_dt>="2020-12-7" and finish_dt<="2021-01-4"')

,name,regions,start_dt,finish_dt


Во время теста проводилась  одна Рождественская маркетинговая акция.

Соединим  таблицы 

`final_ab_events`, 
`new_ab_participants` и `new_final_ab_new_users` в одну.

In [82]:
final_ab_events_participants=new_ab_participants.merge(final_ab_events, on='user_id', how='left')
final_ab_events_participants

,user_id,group,ab_test,event_dt,event_name,details
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,purchase,99.99
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-25 00:04:56,purchase,4.99
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:29,product_cart,NaN
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-25 00:04:57,product_cart,NaN
4,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,product_page,NaN
...,...,...,...,...,...,...
27719,6715343AFBA285AE,B,recommender_system_test,2020-12-07 10:12:15,login,NaN
27720,6715343AFBA285AE,B,recommender_system_test,2020-12-08 22:51:16,login,NaN
27721,6715343AFBA285AE,B,recommender_system_test,2020-12-09 02:28:03,login,NaN
27722,6715343AFBA285AE,B,recommender_system_test,2020-12-10 22:55:14,login,NaN


In [83]:
final_df=final_ab_events_participants.merge(new_final_ab_new_users, on='user_id', how='left')
final_df

,user_id,group,ab_test,event_dt,event_name,details,first_date,region,device
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,purchase,99.99,2020-12-07,EU,PC
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-25 00:04:56,purchase,4.99,2020-12-07,EU,PC
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:29,product_cart,NaN,2020-12-07,EU,PC
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-25 00:04:57,product_cart,NaN,2020-12-07,EU,PC
4,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,product_page,NaN,2020-12-07,EU,PC
...,...,...,...,...,...,...,...,...,...
27719,6715343AFBA285AE,B,recommender_system_test,2020-12-07 10:12:15,login,NaN,2020-12-07,CIS,Android
27720,6715343AFBA285AE,B,recommender_system_test,2020-12-08 22:51:16,login,NaN,2020-12-07,CIS,Android
27721,6715343AFBA285AE,B,recommender_system_test,2020-12-09 02:28:03,login,NaN,2020-12-07,CIS,Android
27722,6715343AFBA285AE,B,recommender_system_test,2020-12-10 22:55:14,login,NaN,2020-12-07,CIS,Android


In [84]:
#проверим таблицу на пропуски:
final_df.isna().sum()

user_id       0    
group         0    
ab_test       0    
event_dt      3026 
event_name    3026 
details       24393
first_date    0    
region        0    
device        0    
dtype: int64

Удалим из таблицы пропущенные значения из столбцов event_dt, event_name.

In [85]:
final_df=final_df=final_df.dropna(subset = ['event_dt','event_name'])

In [86]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24698 entries, 0 to 27723
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     24698 non-null  object        
 1   group       24698 non-null  object        
 2   ab_test     24698 non-null  object        
 3   event_dt    24698 non-null  object        
 4   event_name  24698 non-null  object        
 5   details     3331 non-null   float64       
 6   first_date  24698 non-null  datetime64[ns]
 7   region      24698 non-null  object        
 8   device      24698 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 1.9+ MB


In [87]:
#проверим сколько уникальных пользователей, для проведения теста осталось:
final_df['user_id'].nunique()

3675

События, которые были совершены после 14 дней с момента  регистрации пользователя, уберем из анализа.

In [88]:
#Добавим столбец с днем
final_df['day'] = pd.to_datetime(final_df['event_dt']).dt.date
final_df.head(4)

/tmp/ipykernel_105/3116717443.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,user_id,group,ab_test,event_dt,event_name,details,first_date,region,device,day
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,purchase,99.99,2020-12-07,EU,PC,2020-12-07
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-25 00:04:56,purchase,4.99,2020-12-07,EU,PC,2020-12-25
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:29,product_cart,NaN,2020-12-07,EU,PC,2020-12-07
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-25 00:04:57,product_cart,NaN,2020-12-07,EU,PC,2020-12-25


In [89]:
final_df['day'] = pd.to_datetime(final_df['day'], format='%Y-%m-%d')

/tmp/ipykernel_105/528414635.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [90]:
final_df['lifetime']=final_df['day']-final_df['first_date']
final_df =final_df.query('lifetime < "14days"')
final_df['user_id'].nunique()

/tmp/ipykernel_105/2731774044.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



3675

#### Вывод 

Очистили данные согласно заявленному ТЗ.Ожидаемое количество участников теста: 6000, После очистки данных осталось 3675, что на 61% меньше от плана.Можно уже сделать  заключение о некорректности проведения теста.

<a id='3'></a>
### Исследовательский анализ данных 

- Количество событий на пользователя одинаково распределены в выборках?
- Как число событий в выборках распределено по дням?
- Как меняется конверсия в воронке в выборках на разных этапах?
- Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

#### Количество событий на пользователя одинаково распределены в выборках

In [91]:
final_df['event_name'].value_counts().to_frame()

,event_name
login,10794
product_page,6684
purchase,3232
product_cart,3146


In [92]:
gr_A=final_df.query('group=="A"').groupby(['user_id','event_name'])['event_name'].count().to_frame().rename(columns={'event_name':'count'}).reset_index().sort_values('count')
gr_A

,user_id,event_name,count
1451,3B62EE83708770A5,purchase,1
5854,F0C5FD9E6F7968EC,purchase,1
5853,F0C5FD9E6F7968EC,product_page,1
5852,F0C5FD9E6F7968EC,login,1
3028,7DC0F01827866E41,product_page,1
...,...,...,...
4548,B8EF6F0325A9979F,login,7
2351,6022892C8C3CE8EC,product_page,7
4395,B36555EE73132491,login,7
6018,F8185432C71D445E,login,7


In [93]:
gr_B=final_df.query('group=="B"').groupby(['user_id','event_name'])['event_name'].count().to_frame().rename(columns={'event_name':'count'}).reset_index().sort_values('count')
gr_B

,user_id,event_name,count
1586,CD668979C54E22F7,purchase,1
1015,87B112DF92E5A3BA,login,1
1322,AD6416B175EC960F,product_cart,1
1321,AD6416B175EC960F,login,1
1511,C57EE2E8FCB5C551,login,1
...,...,...,...
1612,D182471A77C0DC13,login,7
1037,89545C7F903DBA34,product_page,7
1038,89545C7F903DBA34,purchase,7
110,115EBC1CA027854A,product_cart,7


In [94]:
gr_A=gr_A.groupby('event_name')['count'].mean().to_frame().reset_index().round(2)
gr_B=gr_B.groupby('event_name')['count'].mean().to_frame().reset_index().round(2)

In [95]:
fig = go.Figure(data=[
    go.Bar(name='Gr_A', x=gr_A['event_name'].to_list(),
           y=gr_A['count'].to_list(),
           text=gr_A['count'].to_list(),
           textposition='auto'),
    go.Bar(name='Gr_В', x=gr_B['event_name'].to_list(),
           y=gr_B['count'].to_list(),
           text=gr_B['count'].to_list(),
           textposition='auto')
])

fig.update_layout(barmode='group', title='График  кол-ва событий на пользователя')

fig.show()

В группе А в среднем на одного пользователя приходится 3 события, в группе В- 2 события.

In [96]:
events_per_user=final_df \
    .groupby(['group', 'event_name']) \
    .agg({'user_id':'nunique'}) \
    .reset_index() \
    .sort_values('user_id', ascending=False)
events_per_user

,group,event_name,user_id
0,A,login,2747
2,A,product_page,1780
4,B,login,927
3,A,purchase,872
1,A,product_cart,824
6,B,product_page,523
7,B,purchase,256
5,B,product_cart,255


In [97]:
fig = px.bar(
    events_per_user, x = 'group', y = 'user_id', text = 'user_id',color = 'event_name', barmode = 'group', title='График распределения количества событий по группам'
)
fig.update_xaxes()
fig.show() 

Судя по графикам, в группе А событий больше чем в группе В.

#### Как число событий в выборках распределено по дням.

In [98]:
day_events=final_df \
    .groupby(['day','group'])['event_name'] \
    .count() \
    .to_frame() \
    .sort_values('event_name', ascending=False) \
    .reset_index() \
    .rename(columns={'event_name':'count'})

fig = px.bar(
    day_events,
    x='day',
    y='count',
    color='group',
    title='Распределение событий в двух группах по дням',
)
fig.update_xaxes()
fig.show()

По графику можно заметить, что 21.12.2020 было совершено больше всего событий ,в группе А -2011,в группе В -430.Также видим, что после 21.12.2020 кол-во событий снижается.

Меньше всего событий было совершено 13.12.2020 в группе А -328,в группе В -176.

####  Как меняется конверсия в воронке в выборках на разных этапах.

In [99]:
final_df

,user_id,group,ab_test,event_dt,event_name,details,first_date,region,device,day,lifetime
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,purchase,99.99,2020-12-07,EU,PC,2020-12-07,0 days
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:29,product_cart,NaN,2020-12-07,EU,PC,2020-12-07,0 days
4,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,product_page,NaN,2020-12-07,EU,PC,2020-12-07,0 days
6,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,login,NaN,2020-12-07,EU,PC,2020-12-07,0 days
8,A7A3664BD6242119,A,recommender_system_test,2020-12-20 15:46:06,product_page,NaN,2020-12-20,EU,iPhone,2020-12-20,0 days
...,...,...,...,...,...,...,...,...,...,...,...
27719,6715343AFBA285AE,B,recommender_system_test,2020-12-07 10:12:15,login,NaN,2020-12-07,CIS,Android,2020-12-07,0 days
27720,6715343AFBA285AE,B,recommender_system_test,2020-12-08 22:51:16,login,NaN,2020-12-07,CIS,Android,2020-12-08,1 days
27721,6715343AFBA285AE,B,recommender_system_test,2020-12-09 02:28:03,login,NaN,2020-12-07,CIS,Android,2020-12-09,2 days
27722,6715343AFBA285AE,B,recommender_system_test,2020-12-10 22:55:14,login,NaN,2020-12-07,CIS,Android,2020-12-10,3 days


In [100]:
final_df['event_name'].value_counts()

login           10794
product_page    6684 
purchase        3232 
product_cart    3146 
Name: event_name, dtype: int64

In [101]:
#создадим сводные таблицы по двум группам и посчитаем кол-во уникальных пользователей на каждом шаге
group_A=final_df \
    .query('group=="A"') \
    .groupby(['event_name'])['user_id'] \
    .nunique() \
    .to_frame() \
    .sort_values('user_id', ascending=False) \
    .reset_index() \
    .rename(columns={'user_id':'count'})

group_B=final_df \
    .query('group=="B"') \
    .groupby(['event_name'])['user_id'] \
    .nunique() \
    .to_frame() \
    .sort_values('user_id', ascending=False) \
    .reset_index() \
    .rename(columns={'user_id':'count'})

In [102]:
#построим воронку событий
fig = go.Figure()
fig.add_trace(go.Funnel(
    name='Group A',
    y = ['login','product_page','product_cart','purchase'],
    x = group_A['count'],
    textinfo = "value+percent previous+ percent initial"
    ))
fig.add_trace(go.Funnel(
    name='Group B',
    y = ['login','product_page','product_cart','purchase'],
    x = group_B['count'],
    textinfo = "value+percent previous+ percent initial"
    )) 
fig.update_layout(title_text='Воронка по группам А/В')

fig.show()

#### Воронка имеет вид 
login-product_page-product_cart-purchase.

До покупки в группе А доходят 49% пользователей от предыдущего шага, в группе В также-49%.
По ТЗ ожидаемый эффект: за 14 дней с момента регистрации улучшение каждой метрики не менее, чем на 10%. Судя по воронке, улучшение метрик в группе В не наблюдается.


#### Вывод 

- Аудитория с конкурирующем тестом пересекалась, чего должно быть не должно.
- Ожидаемое количество участников теста: 6000, после очистки данных осталось 3675, пользователи EU составили менее 15%, фактически это 14%.
- Тест  выпал на время  маркетинговой акции, что еще раз доказывает в некорректности проведения теста.
- в группе А событий больше чем в группе В.
- по графику распределение событий в двух группах по дням, заметен резкий скачок 21.12.2020,скорее всего данный всплеск вызван перед рождественскими праздниками.
- по воронке событий до покупки в двух группах доходят 49% пользователей, ожидаемого эффекта:  улучшение каждой метрики на 10% не наблюдается.


<a id='4'></a>
### Оценить результаты A/B-тестирования

 - Что можно сказать про результаты A/В-тестирования?
 - Проверить статистическую разницу долей z-критерием.

Результатам теста доверять нельзя.

#### Статистическая разница долей z-критерий

Определим гипотезы:

Нулевая гипотеза - H₀: статистически значимых различий в конверсии между группами нет

Альтернативная гипотеза - H₁: статистически значимые различия в конверсии между группами есть

In [103]:
final_df.groupby('group')['user_id'].nunique().to_frame()

,user_id
group,
A,2747
B,928


In [104]:
users_by_events = final_df.pivot_table(
    index='event_name', 
    columns='group', 
    values='user_id',
    aggfunc='nunique') \
    .sort_values(by='A', ascending=False) 
    
users_by_events=users_by_events.reset_index()
users_by_events



group,event_name,A,B
0,login,2747,927
1,product_page,1780,523
2,purchase,872,256
3,product_cart,824,255


In [105]:
users_by_events

group,event_name,A,B
0,login,2747,927
1,product_page,1780,523
2,purchase,872,256
3,product_cart,824,255


In [106]:
exp_table=final_df.groupby('group')['user_id'].nunique() #кол-во пользователей в каждой группе эксперимента
exp_table

group
A    2747
B    928 
Name: user_id, dtype: int64

In [107]:
#Добавим в таблицу users_by_events доли пользователей по каждому событию в каждой группе :
users_by_events['funnel_A'] = (users_by_events['A'] / exp_table['A'])*100
users_by_events['funnel_B'] = (users_by_events['B'] / exp_table['B'])*100
users_by_events

group,event_name,A,B,funnel_A,funnel_B
0,login,2747,927,100.000000,99.892241
1,product_page,1780,523,64.797961,56.357759
2,purchase,872,256,31.743720,27.586207
3,product_cart,824,255,29.996360,27.478448


In [108]:
def z_test(group1, group2, alpha):
    for i in users_by_events.index:
        p1 = users_by_events[group1][i] / exp_table[group1]
        # пропорция успехов во второй группе:
        p2 = users_by_events[group2][i] / exp_table[group2]
        # пропорция успехов в комбинированном датасете:
        p_combined = ((users_by_events[group1][i] + users_by_events[group2][i]) / 
                      (exp_table[group1] + exp_table[group2]))
        # разница пропорций в датасетах
        difference = p1 - p2
        # считаем статистику в ст.отклонениях стандартного нормального распределения
        z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * 
                                        (1/exp_table[group1] + 1/exp_table[group2]))
        # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
        distr = st.norm(0, 1) 
        p_value = (1 - distr.cdf(abs(z_value))) * 2
        print('{} p-значение: {}'.format(users_by_events['event_name'][i], p_value))
        if (p_value < alpha):
            print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
        else:
            print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")

        
z_test('A','B', 0.05)

login p-значение: 0.08529860212027773
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
product_page p-значение: 4.310980554755872e-06
Отвергаем нулевую гипотезу: между долями есть значимая разница
purchase p-значение: 0.017592402663314743
Отвергаем нулевую гипотезу: между долями есть значимая разница
product_cart p-значение: 0.14534814557238196
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


# Вывод 

После анализа данных и результатов A/B теста можно сказать, что тест был проведен некорректно.

- Аудитория с конкурирующем тестом пересекалась, чего должно быть не должно.
- Ожидаемое количество участников теста: 6000, после очистки данных осталось 3675, пользователи EU составили менее 15%, фактически это 14%.
- Тест выпал на время маркетинговой акции, что еще раз доказывает в некорректности проведения теста.
- Некоторые пользователи учавствовали в других тестированиях, что тоже некорректно.
- Количество пользователей в группах значительно отличается.
- В группе Б конверсия ниже, ожидаемого увеличения конверсии на 10 % не получили.

Тестирование стоит завершить, и запустить заново , когда все пункты по ТЗ будут удовлетворены.